# Data Collection

This notebook is responsible for collecting historical Formula 1 race data using the Ergast Developer API.
The collected data will be stored as raw CSV files and will not be modified in this step.


In [5]:
import pandas as pd 


In [6]:
import sys
print(sys.executable)

c:\Users\DELL\AppData\Local\Programs\Python\Python314\python.exe


In [7]:
drivers = pd.read_csv("C:/Users/DELL/Downloads/archive (11)/drivers.csv")
drivers.head()

,driver_id,givenName,familyName,nationality,dob
0,farina,Nino,Farina,Italian,1906-10-30
1,fagioli,Luigi,Fagioli,Italian,1898-06-09
2,reg_parnell,Reg,Parnell,British,1911-07-02
3,cabantous,Yves,Cabantous,French,1904-10-08
4,rosier,Louis,Rosier,French,1905-11-05


In [8]:
drivers.tail()

,driver_id,givenName,familyName,nationality,dob
611,antonelli,Andrea Kimi,Antonelli,Italian,2006-08-25
612,lawson,Liam,Lawson,New Zealander,2002-02-11
613,bortoleto,Gabriel,Bortoleto,Brazilian,2004-10-14
614,doohan,Jack,Doohan,Australian,2003-01-20
615,hadjar,Isack,Hadjar,French,2004-09-28


In [9]:
results = pd.read_csv("c:/Users/DELL/Downloads/archive (11)/results.csv")
results.tail()

,race_id,driver_id,constructor_id,grid,position,position_order,points,laps,status
7595,2025_5,stroll,aston_martin,16,16,16,0.0,49,Lapped
7596,2025_5,doohan,alpine,17,17,17,0.0,49,Lapped
7597,2025_5,bortoleto,sauber,20,18,18,0.0,49,Lapped
7598,2025_5,tsunoda,red_bull,8,R,19,0.0,1,Retired
7599,2025_5,gasly,alpine,9,R,20,0.0,0,Retired


In [10]:
constructors = pd.read_csv("c:/Users/DELL/Downloads/archive (11)/constructors.csv")
constructors.tail()

,constructor_id,name,nationality,Wikipedia_url
163,haas,Haas F1 Team,American,NaN
164,racing_point,Racing Point,British,NaN
165,alphatauri,AlphaTauri,Italian,NaN
166,alpine,Alpine F1 Team,French,NaN
167,rb,RB F1 Team,Italian,NaN


In [11]:
races = pd.read_csv("c:/Users/DELL/Downloads/archive (11)/races.csv")
races.tail()

,race_id,season,round,race_name,date,time,circuit_id
1144,2025_20,2025,20,Mexico City Grand Prix,2025-10-26,20:00:00Z,rodriguez
1145,2025_21,2025,21,São Paulo Grand Prix,2025-11-09,17:00:00Z,interlagos
1146,2025_22,2025,22,Las Vegas Grand Prix,2025-11-23,04:00:00Z,vegas
1147,2025_23,2025,23,Qatar Grand Prix,2025-11-30,16:00:00Z,losail
1148,2025_24,2025,24,Abu Dhabi Grand Prix,2025-12-07,13:00:00Z,yas_marina


In [12]:
quali = pd.read_csv("c:/Users/DELL/Downloads/archive (11)/qualifying.csv")
quali.tail()

,race_id,driver_id,constructor_id,position,q1,q2,q3
3012,2025_5,stroll,aston_martin,16,1:28.645,NaN,NaN
3013,2025_5,doohan,alpine,17,1:28.739,NaN,NaN
3014,2025_5,hulkenberg,sauber,18,1:28.782,NaN,NaN
3015,2025_5,ocon,haas,19,1:29.092,NaN,NaN
3016,2025_5,bortoleto,sauber,20,1:29.462,NaN,NaN


In [13]:
results['position'] = pd.to_numeric(results['position'], errors = 'coerce')
driver_stats = results.groupby('driver_id').agg({
    'position':['mean','std'],
    'points':'sum',
    'status': lambda x: sum('DNF' is s for s in x)
}).reset_index()

driver_stats.columns = ['driverId','avg_position','std_position','total_points','dnfs']
driver_stats.head()

<>:5: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
<>:5: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
C:\Users\DELL\AppData\Local\Temp\ipykernel_16956\2702328534.py:5: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  'status': lambda x: sum('DNF' is s for s in x)


,driverId,avg_position,std_position,total_points,dnfs
0,abate,NaN,NaN,0.0,0
1,abecassis,NaN,NaN,0.0,0
2,adamich,7.200000,3.701351,6.0,0
3,ader,22.000000,NaN,0.0,0
4,agabashian,7.333333,4.163332,1.5,0


In [14]:
driver_stats['std_position'] = driver_stats['std_position'].fillna(0)

In [15]:
driver_stats['avg_position'] = driver_stats['avg_position'].fillna(0)


In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

features_to_scale = ['avg_position', 'std_position', 'dnfs']

driver_stats[features_to_scale] = scaler.fit_transform(
    driver_stats[features_to_scale]
)


In [17]:
driver_stats['DCI'] = (
    (1 - driver_stats['avg_position']) * 0.4 +
    (1 - driver_stats['std_position']) * 0.3 +
    (1 - driver_stats['dnfs']) * 0.3
)


In [21]:

drivers = pd.read_csv("C:/Users/DELL/Downloads/archive (11)/drivers.csv")
driver_stats = driver_stats.merge(
    drivers[['driver_id', 'familyName']],
    on='driver_id',
    how='left'
)

driver_stats = driver_stats.sort_values(by='DCI', ascending=False)

print(driver_stats[['surname', 'DCI']].head(10))


KeyError: 'driver_id'

In [22]:
print(drivers.columns)

Index(['driver_id', 'givenName', 'familyName', 'nationality', 'dob'], dtype='str')


In [23]:
driver_stats.columns

Index(['driverId', 'avg_position', 'std_position', 'total_points', 'dnfs',
       'DCI'],
      dtype='str')

In [24]:
driver_stats = driver_stats.rename(columns={'driverId': 'driver_id'})


In [25]:
driver_stats.columns

Index(['driver_id', 'avg_position', 'std_position', 'total_points', 'dnfs',
       'DCI'],
      dtype='str')

In [26]:
drivers = pd.read_csv("C:/Users/DELL/Downloads/archive (11)/drivers.csv")

# Create full name
drivers['full_name'] = drivers['givenName'] + " " + drivers['familyName']

driver_stats = driver_stats.merge(
    drivers[['driver_id', 'full_name']],
    on='driver_id',
    how='left'
)

driver_stats = driver_stats.sort_values(by='DCI', ascending=False)

print(driver_stats[['full_name', 'DCI']].head(10))


           full_name  DCI
358      Mike Magill  1.0
348  Ricardo Londoño  1.0
341  Bayliss Levrett  1.0
372       Tony Marsh  1.0
49        Derek Bell  1.0
52         Tom Belsø  1.0
351      Roger Loyer  1.0
349    Henri Louveau  1.0
309        Joe Kelly  1.0
311     Danny Kladis  1.0


In [29]:

races.columns

Index(['race_id', 'season', 'round', 'race_name', 'date', 'time',
       'circuit_id'],
      dtype='str')

In [31]:
races

,race_id,season,round,race_name,date,time,circuit_id
0,1950_1,1950,1,British Grand Prix,1950-05-13,NaN,silverstone
1,1950_2,1950,2,Monaco Grand Prix,1950-05-21,NaN,monaco
2,1950_3,1950,3,Indianapolis 500,1950-05-30,NaN,indianapolis
3,1950_4,1950,4,Swiss Grand Prix,1950-06-04,NaN,bremgarten
4,1950_5,1950,5,Belgian Grand Prix,1950-06-18,NaN,spa
...,...,...,...,...,...,...,...
1144,2025_20,2025,20,Mexico City Grand Prix,2025-10-26,20:00:00Z,rodriguez
1145,2025_21,2025,21,São Paulo Grand Prix,2025-11-09,17:00:00Z,interlagos
1146,2025_22,2025,22,Las Vegas Grand Prix,2025-11-23,04:00:00Z,vegas
1147,2025_23,2025,23,Qatar Grand Prix,2025-11-30,16:00:00Z,losail


In [30]:
results.columns

Index(['race_id', 'driver_id', 'constructor_id', 'grid', 'position',
       'position_order', 'points', 'laps', 'status'],
      dtype='str')

In [32]:
results = pd.read_csv("C:/Users/DELL/Downloads/archive (11)/results.csv")
races = pd.read_csv("C:/Users/DELL/Downloads/archive (11)/races.csv")

# Clean column names
results.columns = results.columns.str.strip()
races.columns = races.columns.str.strip()

# Merge to get season year
results = results.merge(
    races[['race_id', 'season']],
    on='race_id',
    how='left'
)

print(results[['race_id', 'season']].head())


  race_id  season
0  1950_1    1950
1  1950_1    1950
2  1950_1    1950
3  1950_1    1950
4  1950_1    1950


In [33]:
recent_results = results[results['season'] >= 2022]
recent_results['season'].unique()

array([2022, 2023, 2024, 2025])

In [34]:
season_weights ={
    2025: 1.0,
    2024: 0.8,
    2023: 0.6,
    2022: 0.4
}
recent_results['season_weights'] = recent_results['season'].map(season_weights)

In [35]:
recent_results['position'] = pd.to_numeric(
    recent_results['position'],
    errors='coerce'
)

In [38]:
weighted_stats = recent_results.groupby('driver_id').apply(
    lambda x: pd.Series({
        'weighted_avg_position': (x['position'] * x['season_weights']).mean(),
        'weighted_points': (x['points'] * x['season_weights']).sum(),
        'dnfs': sum('DNF' in str(s) for s in x['status'])
    })
).reset_index()


In [39]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

features = ['weighted_avg_position', 'dnfs']
weighted_stats[features] = scaler.fit_transform(weighted_stats[features])

weighted_stats['recent_DCI'] = (
    (1 - weighted_stats['weighted_avg_position']) * 0.6 +
    (1 - weighted_stats['dnfs']) * 0.4
)


In [40]:
weighted_stats = weighted_stats.rename(columns={'driverId': 'driver_id'})

drivers = pd.read_csv("C:/Users/DELL/Downloads/archive (11)/drivers.csv")
drivers['full_name'] = drivers['givenName'] + " " + drivers['familyName']

weighted_stats = weighted_stats.merge(
    drivers[['driver_id', 'full_name']],
    on='driver_id',
    how='left'
)

weighted_stats = weighted_stats.sort_values(
    by='recent_DCI',
    ascending=False
)

print(weighted_stats[['full_name', 'recent_DCI']].head(10))


           full_name  recent_DCI
16    Max Verstappen    1.000000
20      Sergio Pérez    0.990204
15   Charles Leclerc    0.958754
23    George Russell    0.918458
18      Lando Norris    0.905392
24      Carlos Sainz    0.902041
10    Lewis Hamilton    0.884425
28  Sebastian Vettel    0.873684
21     Oscar Piastri    0.867055
1    Fernando Alonso    0.850967


In [41]:
# Make sure position numeric
recent_results['position'] = pd.to_numeric(
    recent_results['position'],
    errors='coerce'
)

constructor_stats = recent_results.groupby('constructor_id').apply(
    lambda x: pd.Series({
        'weighted_points': (x['points'] * x['season_weights']).sum(),
        'avg_finish': (x['position'] * x['season_weights']).mean()
    })
).reset_index()


In [42]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

features = ['weighted_points', 'avg_finish']
constructor_stats[features] = scaler.fit_transform(constructor_stats[features])

# Higher points good, lower finish position good
constructor_stats['constructor_strength'] = (
    constructor_stats['weighted_points'] * 0.7 +
    (1 - constructor_stats['avg_finish']) * 0.3
)


In [43]:
latest_constructor = recent_results.sort_values('season').groupby('driver_id').tail(1)

driver_constructor_map = latest_constructor[['driver_id', 'constructor_id']]


In [44]:
# Rename for merge compatibility
weighted_stats = weighted_stats.rename(columns={'driverId': 'driver_id'})
driver_constructor_map = driver_constructor_map.rename(columns={'driverId': 'driver_id'})

# Merge driver -> constructor
final_model_df = weighted_stats.merge(
    driver_constructor_map,
    on='driver_id',
    how='left'
)

# Merge constructor strength
final_model_df = final_model_df.merge(
    constructor_stats[['constructor_id', 'constructor_strength']],
    on='constructor_id',
    how='left'
)


In [45]:
final_model_df['championship_score'] = (
    final_model_df['recent_DCI'] * 0.6 +
    final_model_df['constructor_strength'] * 0.4
)


In [46]:
final_model_df = final_model_df.sort_values(
    by='championship_score',
    ascending=False
)

print(final_model_df[['driver_id', 'recent_DCI', 
                      'constructor_strength', 
                      'championship_score']].head(10))


         driver_id  recent_DCI  constructor_strength  championship_score
0   max_verstappen    1.000000              1.000000            1.000000
1            perez    0.990204              1.000000            0.994122
2          leclerc    0.958754              0.752395            0.876210
17         tsunoda    0.761319              1.000000            0.856792
4           norris    0.905392              0.750790            0.843551
6         hamilton    0.884425              0.752395            0.831613
8          piastri    0.867055              0.750790            0.820549
3          russell    0.918458              0.645918            0.809442
12       antonelli    0.811815              0.645918            0.745456
7           vettel    0.873684              0.355313            0.666336


In [47]:
final_model_df.to_csv("../data/final_model_data.csv", index=False)
